Scan for axion in radiation dominated Uinverse.

The scan proceeds as follows:

The for each value of $f_a$, we find $\theta_i^{\rm approx}$ such that $\Omega h^2 = 0.12$ assuming that $sin(\theta_i^{\rm approx}) \approx \theta_i^{\rm approx}$. Then, we scan values of $\theta_i$ close to $\theta_i^{\rm approx}$. This results in a range of values of $\theta_i$ for each $f_a$, with $\Omega h^2$ close to the observed value.

In [1]:
import os
curDir=os.getcwd()#this is the directory of the notebook
os.chdir('../../')

from interfacePy.Axion import Axion 
from interfacePy.Cosmo import h_hub,T0,rho_crit,relicDM_obs,s 
from interfacePy.AxionMass import ma2 

os.chdir(curDir)

In [2]:
import numpy as np

import matplotlib
#matplotlib.use('WebAgg')
#matplotlib.use('Qt4Cairo')
#matplotlib.use('Qt5Cairo')
matplotlib.use('nbAgg')
import matplotlib.pyplot as plt

import subprocess
import sys
import time
from pathlib import Path
import datetime
from IPython.display import clear_output


plt.rcParams['font.family']='serif'
plt.rcParams['font.size']=14
plt.rcParams['mathtext.fontset']='stix'

In [3]:
class scan:
    def __init__(self,cpus,table_fa,len_theta,tmax,TSTOP,ratio_ini,N_convergence_max,convergence_lim,inputFile,
                break_after=0,break_time=60,break_command=''):
        '''
        scan for different values of fa (in table_fa) and find the theta_i closer to reloc_obs.
        
        Comment 1: The way it works is the following:
        1) we solve for theta_i=1e-5.
        2) based on this, rescale it in order to find an appropriate theta_i such that Omegah^2 = relic_obs
        3) use this rescaled theta_i as initial point, and scan for len_theta between 
            np.linspace(min([theta_i*0.85,1]),min([theta_i*1.2,np.pi]),len_theta)
        4) finally, the point with Omegah^2 closer to relic_obs is stored.
        
        Comment 2: If the scan exits before it finishes, it will continue from the point it stopped.
        In order to start from the beginning, delete the file "count.dat".
        
        
        cpus: number of points to run simultaneously (No of cpus available). 
        table_fa: table of fa to scan
        len_theta: number of points to search for theta closest to relic_obs. This search happens in 
        np.linspace(min([theta_i*0.85,1]),min([theta_i*1.2,np.pi]),self.len_theta), with theta_i is the angle
        that results in Omega h^2=relic_obs assuming theta_i<<1.

        tmax: if t>tmax the integration stops (rempember that t=log(a/a_i))
        TSTOP: if the temperature drops below this, integration stops
        ratio_ini: integration starts when 3H/m_a<~ratio_ini (this is passed to AxionEOM, 
        in order to make the interpolations start at this point)

        N_convergence_max and convergence_lim: integration stops after the adiabatic invariant 
        hasn't changed more than convergence_lim% for N_convergence_max consecutive peaks
        
        inputFile: file that describes the cosmology. the columns should be: t T[GeV] logH     
        
        break_after,break_time: take a break after break_after seconds for break_time seconds
        break_command: before it takes a break, run this system command (this may be a script to send the results
        to an e-mail, or back them up)
        '''
        
        self.cpus=cpus
        self.Table_fa=table_fa
        self.len_theta=len_theta
        
        self.tmax=tmax
        self.TSTOP=TSTOP
        self.ratio_ini=ratio_ini
        self.Npeaks=N_convergence_max
        self.conv=convergence_lim
        self.inputFile=inputFile

        
        self.break_after=break_after        
        self.break_time=break_time
        self.break_command=break_command
        
        
        self.FileDate=datetime.datetime.now()
        self.FileName = "{}".format(self.FileDate.strftime('%d-%m-%Y_%H-%M-%S')) 
        self._p = Path(self.FileName+'.dat')
        
        
        self.in_file="in.xtx"
        self.count_file="count.xtx"
    def run_batch(self):
        '''
        run a batch.
        '''
        # get the result     
        points=subprocess.check_output( [r"./parallel_scan",  r"../Cpp/AxionExample/AxionExample.run", str(self.cpus),self.in_file]).decode(sys.stdout.encoding)
        points=points.split('\n')
        

        points=np.array([np.array(i.split(),np.float64)  for i in points[:-1] ])
        points=np.array(sorted(points, key=lambda arr: arr[0]))
        absDiff=np.argmin(np.abs(points[:,4]-relicDM_obs))
        
        File= self._p.open('ab')            
        np.savetxt(File,np.array([points[absDiff]]))

        File.close()

    def run(self):

        try:
            with open(self.count_file,'r') as _:
                last_batch=int(_.read())
        except:
            last_batch=0
            
        Total_batches=len(self.Table_fa)-last_batch
            
        totalT=0
        meanT=0
        ETA=0
        batch=0
        
        sleepTimer=0
        for fa in self.Table_fa[last_batch:]:
            time0=time.time()

            ########################---find theta_i (assuming theta_i<<1) such that Omega h^=0.12---########################

            theta_small=1e-5
            ax=Axion(theta_small,fa,self.tmax,self.TSTOP,self.ratio_ini,self.Npeaks,self.conv,self.inputFile)

            ax.solve()
            ax.getPeaks()
            
            T=ax.T_peak[-1]
            theta=ax.theta_peak[-1]

            relic=ax.relic
            theta_obs=np.sqrt(theta**2/relic*relicDM_obs)

            relic=s(T0)/s(T)*0.5*np.sqrt(ma2(0,1)*ma2(T,1))*theta_obs**2*h_hub**2/rho_crit

            theta_small_i=theta_small*theta_obs/theta
            if theta_small_i>np.pi:
                theta_small_i=np.pi

            Table_theta_i=np.linspace(min([theta_small_i*0.85,1]),min([theta_small_i*1.2,np.pi]),self.len_theta)

            del ax
            ########################---END---########################

            file=open(self.in_file , 'w')
            for theta_i in Table_theta_i :
                file.write( '{0} {1} {2} {3} {4} {5} {6} {7} \n'.format(theta_i, fa, self.tmax, self.TSTOP, self.ratio_ini, self.Npeaks, self.conv, self.inputFile) )
            file.close()
            
            self.run_batch()
            
            batch+=1
            totalT+=time.time()-time0
            meanT=totalT/batch

            if self.break_after>0:
                sleepTimer+=time.time()-time0
                if sleepTimer>self.break_after:
                    os.system(self.break_command)
                    print("taking a break")
                    time.sleep(self.break_time)
                    clear_output(wait=True)
                    sleepTimer=0
                
            ETA=meanT*(Total_batches-batch)
            print('======================================== \n',
            'Completed batches:  ',batch,' out of ', Total_batches ,'\n',
            'Running for:     {0:.50s}'.format( str(datetime.timedelta(seconds=totalT))  ), '\n', 
            'ETA:             {0:.50s}'.format(str(datetime.timedelta(seconds=ETA) )  ),'\n', 
            'Time per batch:  {0:.3f} sec'.format(meanT),
            '\n========================================' )
            clear_output(wait=True)
            
            with open(self.count_file,'w') as _:
                _.write(str(batch+last_batch))

        os.remove(self.in_file)
        os.remove(self.count_file)
        print('Done!')
        
    def readData(self):
        self.points=np.loadtxt(self.FileName+'.dat')


In [4]:
fa=np.logspace(9,20,60)

Scan=scan(8,fa,200,
    tmax=500,
    TSTOP=1e-4,
    ratio_ini=1e3,
    N_convergence_max=5,
    convergence_lim=1e-2,
    inputFile="../InputExamples/RDinput.dat",#"../InputExamples/NSCinput.dat"
    break_after=10,
    break_time=5,
    break_command=''
)

In [5]:
Scan.run()

KeyboardInterrupt: 

In [ ]:
Scan.readData()

In [ ]:
len(Scan.points)

In [ ]:
ch=Scan.points[:,4]<relicDM_obs+0.01
ch=np.logical_and(ch,Scan.points[:,4]>relicDM_obs-0.01)

In [ ]:
fig=plt.figure(figsize=(8,3))
fig.subplots_adjust(bottom=0.15, left=0.15, top = 0.85, right=0.9,wspace=0.0,hspace=0.0)
fig.suptitle('')

sub = fig.add_subplot(1,1,1)

sub.scatter(Scan.points[ch][:,1], Scan.points[ch][:,0], alpha=1, c='xkcd:black', s=20, marker='+',label=r'$\theta_{\rm ini}$')


sub.legend(bbox_to_anchor=(0.98, 0.95),borderaxespad=0., 
           borderpad=0,ncol=1,loc='upper right',fontsize=15,framealpha=0)


sub.set_xlabel(r'$f_{\alpha}~[{\rm GeV}]$')
sub.xaxis.set_label_coords(0.5, -0.1)
sub.set_ylabel(r'$\theta$')
sub.yaxis.set_label_coords(-0.1,0.5)
 
sub.set_xscale('log')
sub.set_yscale('log')



# fig.savefig('fa_vs_theta_i.pdf',bbox_inches='tight')
fig.show()